In [2]:
%load_ext autoreload
%autoreload 2

import qdrant_client
from utils import get_milelion_urls, extract_milelion
import glob
import os

from crawl4ai.extraction_strategy import LLMExtractionStrategy, CosineStrategy
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core import Settings, StorageContext, VectorStoreIndex, PromptTemplate, get_response_synthesizer
from llama_index.core.readers.json import JSONReader
from llama_index.llms.openai import OpenAI
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine


/Users/ivankoh/miniconda/envs/rag-any-website/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# process the urls - take 
def get_milelion_urls_for_year(year: int) -> list[str]:
    all_urls = get_milelion_urls()
    year_urls = [url for url in all_urls if f"/{year}/" in url and not url.endswith((".jpg", ".png", ".webp", ".jpeg"))]
    return year_urls

urls_2024 = get_milelion_urls_for_year(2024)
print(len(urls_2024))
print(urls_2024)

Getting Sitemap for https://milelion.com/post-sitemap1.xml
Found 16007 links
594
['https://milelion.com/2024/10/17/qatar-airways-offering-up-to-60-bonus-on-avios-purchases/', 'https://milelion.com/2024/10/18/accor-plus-member-for-a-day-how-it-works/', 'https://milelion.com/2024/10/18/what-happens-to-my-credit-card-points-when-i-die/', 'https://milelion.com/2024/10/18/milelion-weekly-deal-summary-12-18-oct-24/', 'https://milelion.com/2024/07/11/world-of-hyatt-offering-2x-points-at-hyatt-place-and-hyatt-house/', 'https://milelion.com/2024/10/17/review-maybank-horizon-visa-signature-card/', 'https://milelion.com/2024/01/09/whats-the-best-credit-card-for-hospital-or-medical-bills/', 'https://milelion.com/2024/10/16/2024-edition-best-credit-cards-for-pharmacies/', 'https://milelion.com/2024/10/16/full-list-of-priority-pass-dragonpass-restaurants-at-changi-airport/', 'https://milelion.com/2024/10/10/dcs-flex-card-review-weird-flex-but-ok/', 'https://milelion.com/2024/10/15/krisflyer-spontane

In [ ]:
# chunk the urls in batches of 10
def chunk_list(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

url_chunks = list(chunk_list(urls_2024, 10))

llm_extraction_strategy = LLMExtractionStrategy(
        provider="openai/gpt-4o",
        api_token=os.getenv('OPENAI_API_KEY'),
        instruction=(
            "From the crawled content, extract the following details: "
            "1. Title of the article "
            "2. Summary of the article, which is a detailed summary "
            "3. Brief summary of the article, which is a paragraph text "
            "4. Keywords related to the article, which is a list of keywords. "
            "Exclude any advertisement or noisy content. "
            'The extracted JSON format should look like this: '
            '{ "title": "Article Title", "summary": "Detailed summary of the article.", '
            '"brief_summary": "Brief summary in a paragraph.", "keywords": ["keyword1", "keyword2", "keyword3"] }'
        ),
        extraction_type="schema",
        apply_chunking=False
    )

for i, chunk in enumerate(url_chunks):
    print(f"Chunk {i}: {chunk}")
    # await extract_milelion(chunk, llm_extraction_strategy, out_dir='.data/') # expensive


In [6]:
# Initialize Qdrant client
client = qdrant_client.QdrantClient(
    host="localhost",
    port=6333
)

# Create a collection
collection_name = "milelion_docs"
vectors_size = 768  # Dimensions of vectors to store in Qdrant
distance = qdrant_client.models.Distance.COSINE

client.recreate_collection(
    collection_name=collection_name,
    vectors_config=qdrant_client.models.VectorParams(size=vectors_size, distance=distance),
)

# Set up vector store
Settings.embed_model = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")
Settings.llm = OpenAI()
vector_store = QdrantVectorStore(client=client, collection_name=collection_name)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Load JSON and create an index
def load_and_index_json(directory_path):
    reader = JSONReader(
        levels_back=0,
        collapse_length=None,
        ensure_ascii=False,
        is_jsonl=False,
        clean_json=True
    )

    # Find all JSON files in the specified directory
    json_files = glob.glob(os.path.join(directory_path, "*.json"))

    # Load the data from each JSON file
    documents = []
    for json_file in json_files:
        documents.extend(reader.load_data(input_file=json_file, extra_info={
            "source_doc": json_file,
            "source_url": "https://milelion.com/" + json_file.replace(".data/", "").replace(".json", "").replace("_", "/"),
        }))

    # Create an index for querying
    index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
    return index

# Specify the directory containing your JSON files
json_directory = ".data/"

# Load JSON and create an index
index = load_and_index_json(json_directory)
print("Indexing completed!")

/var/folders/dq/33rpc0cx4f54t0vx2cdcgczr0000gn/T/ipykernel_46232/3030254198.py:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(
Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 49461.13it/s]


Indexing completed!


In [9]:
qa_prompt_tmpl = (
    "Context information is below.\n"
    "-------------------------------"
    "{context_str}\n"
    "-------------------------------"
    "Given the context information and not prior knowledge,"
    "answer the query. Please be concise, and complete.\n"
    "Provide the source link of which you got your information from"
    "If the context does not contain an answer to the query,"
    "respond with \"I don't know!\"."
    "Query: {query_str}\n"
    "Answer: "
)
qa_prompt = PromptTemplate(qa_prompt_tmpl)

retriever = VectorIndexRetriever(index=index, similarity_top_k=5, sparse_top_k=10)
response_synthesizer = get_response_synthesizer(
    llm = OpenAI(),
    text_qa_template = qa_prompt
)
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer
)


In [10]:
from pprint import pprint

pprint(query_engine.query(
    """
    I want to buy a ticket to Tokyo. What are the best options for credit cards? Why?
    """
).response)

('The best options for credit cards to buy a ticket to Tokyo would be credit '
 'cards that offer high miles per dollar (mpd) rewards for airline ticket '
 'purchases. Some recommended options include the UOB Lady’s Card, DBS WWMC, '
 'HSBC Revolution, and Maybank Visa Infinite. These cards can help you earn up '
 'to 4 mpd on your air ticket purchase, allowing you to accumulate more miles '
 'for your next trip to Tokyo.\n'
 '\n'
 'Source: '
 'https://milelion.com/2024/06/08/best-credit-cards-for-airline-tickets/')
